In [ ]:
import pickle

file_path = '/Volumes/harris/somnotate/to_score_set/pickle_eeg_signal/eeg_data_sub-010_ses-01_recording-01.pkl'

file = open(file_path, 'rb')
        
eeg_data = pickle.load(file)

import pandas as pd
sleep_df = pd.read_csv('/Volumes/harris/somnotate/to_score_set/vis_back_to_csv/automated_state_annotationoutput_sub-010_ses-01_recording-01_time-0-69h_512Hz.csv')

/var/folders/sy/1pj77pp55jgbyxgsst2fw9rr0000gn/T/ipykernel_42075/1149381519.py:7: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  eeg_data = pickle.load(file)


In [ ]:
import numpy as np
import pandas as pd
import pickle
from neurodsp.spectral import compute_spectrum

def get_nrem_baseline_power(eeg_pickle_path, sleep_stages_csv):
    # Load data
    #with open(eeg_pickle_path, 'rb') as f:
        #eeg_data = pickle.load(f)
    
    #sleep_df = pd.read_csv(sleep_stages_csv)
    
    # Parameters
    fs = 512
    freq_range = (0.5, 50)
    
    # Get NREM segments
    nrem_mask = sleep_df['sleepStage'] == 2
    nrem_indices = np.where(nrem_mask)[0]

    nrem_mask = sleep_stages['sleepStage'] == 2
    continuous_segments = []
    start_idx = None
    for i in range(len(nrem_mask)):
        if nrem_mask[i] and start_idx is None:
            start_idx = i
        elif not nrem_mask[i] and start_idx is not None:
            end_idx = i - 1
            continuous_segments.append((start_idx, end_idx))
            start_idx = None
    if start_idx is not None:
        continuous_segments.append((start_idx, len(nrem_mask) - 1))


    def compute_epoch_power(eeg_data, start_idx, end_idx, fs=512, freq_range=(0.5, 50)):
    eeg1_epoch = eeg_data['EEG1'][start_idx:end_idx+1]
    eeg2_epoch = eeg_data['EEG2'][start_idx:end_idx+1]
    
    f, Pxx_eeg1 = compute_spectrum(eeg1_epoch, fs, method='welch', nperseg=fs*2)
    f, Pxx_eeg2 = compute_spectrum(eeg2_epoch, fs, method='welch', nperseg=fs*2)
    
    freq_mask = (f >= freq_range[0]) & (f <= freq_range[1])
    
    power_eeg1 = np.mean(Pxx_eeg1[freq_mask])
    power_eeg2 = np.mean(Pxx_eeg2[freq_mask])
    
    return power_eeg1, power_eeg2

def compute_baseline_power(eeg_data, sleep_stages, fs=512, freq_range=(0.5, 50)):
    segments = find_continuous_nrem_segments(sleep_stages)
    power_eeg1_list = []
    power_eeg2_list = []
    
    for start_idx, end_idx in segments:
        power_eeg1, power_eeg2 = compute_epoch_power(eeg_data, start_idx, end_idx, fs, freq_range)
        power_eeg1_list.append(power_eeg1)
        power_eeg2_list.append(power_eeg2)
    
    baseline_power_eeg1 = np.mean(power_eeg1_list)
    baseline_power_eeg2 = np.mean(power_eeg2_list)
    
    return baseline_power_eeg1, baseline_power_eeg2

    
    # Initialize arrays for power calculations
    powers_eeg1 = []
    powers_eeg2 = []
    
    # Calculate power for each continuous NREM segment
    for start_idx, end_idx in continuous_segments:
        eeg1_epoch = eeg_data['EEG1'][start_idx:end_idx+1]
        eeg2_epoch = eeg_data['EEG2'][start_idx:end_idx+1]
        
        f, Pxx_eeg1 = compute_spectrum(eeg1_epoch, fs, method='welch', nperseg=fs*2)
        f, Pxx_eeg2 = compute_spectrum(eeg2_epoch, fs, method='welch', nperseg=fs*2)
        
        freq_mask = (f >= freq_range[0]) & (f <= freq_range[1])
        
        power_eeg1 = np.mean(Pxx_eeg1[freq_mask])
        power_eeg2 = np.mean(Pxx_eeg2[freq_mask])
    
    # Average between channels
    mean_power_eeg1 = np.mean(powers_eeg1, axis=0)
    mean_power_eeg2 = np.mean(powers_eeg2, axis=0)
    baseline_power = (mean_power_eeg1 + mean_power_eeg2) / 2
    
    return {
        'frequencies': freqs1,
        'baseline_power': baseline_power,
        'eeg1_power': mean_power_eeg1,
        'eeg2_power': mean_power_eeg2
    }

# Example usage
result = get_nrem_baseline_power(eeg_data, sleep_df)

EEG1 segment shape: (9699328,)
EEG2 segment shape: (9699328,)
freqs1 shape: (100,), power1 shape: (100,)
freqs2 shape: (100,), power2 shape: (100,)
EEG1 segment shape: (0,)
EEG2 segment shape: (0,)


/Users/Volka/miniconda3/envs/fooof_venv/lib/python3.9/site-packages/neurodsp/spectral/power.py:181: UserWarning: nperseg = 1024 is greater than input length  = 0, using nperseg = 0
  freqs, _, spg = spectrogram(sig, fs, window, nperseg, noverlap)
/Users/Volka/miniconda3/envs/fooof_venv/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/Volka/miniconda3/envs/fooof_venv/lib/python3.9/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


IndexError: invalid index to scalar variable.

In [7]:
nrem_mask = sleep_df['sleepStage'] == 2

In [15]:
continuous_segments = []
start_idx = None
for i in range(len(nrem_mask)):
    if nrem_mask[i] and start_idx is None:
        start_idx = i
    elif not nrem_mask[i] and start_idx is not None:
        end_idx = i - 1
        continuous_segments.append((start_idx, end_idx))
        start_idx = None
if start_idx is not None:
    continuous_segments.append((start_idx, len(nrem_mask) - 1))

In [11]:
import pickle
import pandas as pd
import numpy as np
from scipy.signal import welch

def compute_baseline_power(eeg_data, sleep_stages, fs=512, freq_range=(0.5, 50)):
    # Filter the data for NREM sleep stage
    nrem_indices = sleep_stages[sleep_stages['sleepStage'] == 2].index
    eeg1_nrem = eeg_data['EEG1'][nrem_indices]
    eeg2_nrem = eeg_data['EEG2'][nrem_indices]
    
    # Compute power spectral density using Welch's method
    f, Pxx_eeg1 = welch(eeg1_nrem, fs, nperseg=fs*2)
    f, Pxx_eeg2 = welch(eeg2_nrem, fs, nperseg=fs*2)
    
    # Find the indices corresponding to the desired frequency range
    freq_mask = (f >= freq_range[0]) & (f <= freq_range[1])
    
    # Calculate the average power in the specified frequency range
    baseline_power_eeg1 = np.mean(Pxx_eeg1[freq_mask])
    baseline_power_eeg2 = np.mean(Pxx_eeg2[freq_mask])
    
    return baseline_power_eeg1, baseline_power_eeg2

baseline_power_eeg1, baseline_power_eeg2 = compute_baseline_power(eeg_data, sleep_df)
print(f"Baseline NREM Power - EEG1: {baseline_power_eeg1}, EEG2: {baseline_power_eeg2}")

Baseline NREM Power - EEG1: 66.66157940833855, EEG2: 47.02825486666751


In [12]:
def find_continuous_nrem_segments(sleep_stages):
    nrem_mask = sleep_stages['sleepStage'] == 2
    continuous_segments = []
    start_idx = None
    for i in range(len(nrem_mask)):
        if nrem_mask[i] and start_idx is None:
            start_idx = i
        elif not nrem_mask[i] and start_idx is not None:
            end_idx = i - 1
            continuous_segments.append((start_idx, end_idx))
            start_idx = None
    if start_idx is not None:
        continuous_segments.append((start_idx, len(nrem_mask) - 1))
    return continuous_segments

def compute_epoch_power(eeg_data, start_idx, end_idx, fs=512, freq_range=(0.5, 50)):
    eeg1_epoch = eeg_data['EEG1'][start_idx:end_idx+1]
    eeg2_epoch = eeg_data['EEG2'][start_idx:end_idx+1]
    
    f, Pxx_eeg1 = welch(eeg1_epoch, fs, nperseg=fs*2)
    f, Pxx_eeg2 = welch(eeg2_epoch, fs, nperseg=fs*2)
    
    freq_mask = (f >= freq_range[0]) & (f <= freq_range[1])
    
    power_eeg1 = np.mean(Pxx_eeg1[freq_mask])
    power_eeg2 = np.mean(Pxx_eeg2[freq_mask])
    
    return power_eeg1, power_eeg2

def compute_baseline_power(eeg_data, sleep_stages, fs=512, freq_range=(0.5, 50)):
    segments = find_continuous_nrem_segments(sleep_stages)
    power_eeg1_list = []
    power_eeg2_list = []
    
    for start_idx, end_idx in segments:
        power_eeg1, power_eeg2 = compute_epoch_power(eeg_data, start_idx, end_idx, fs, freq_range)
        power_eeg1_list.append(power_eeg1)
        power_eeg2_list.append(power_eeg2)
    
    baseline_power_eeg1 = np.mean(power_eeg1_list)
    baseline_power_eeg2 = np.mean(power_eeg2_list)
    
    
    return baseline_power_eeg1, baseline_power_eeg2

baseline_power_eeg1, baseline_power_eeg2 = compute_baseline_power(eeg_data, sleep_df)
print(f"Baseline NREM Power - EEG1: {baseline_power_eeg1}, EEG2: {baseline_power_eeg2}")

/Users/Volka/miniconda3/envs/fooof_venv/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 1024 is greater than input length  = 512, using nperseg = 512
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


Baseline NREM Power - EEG1: 61.90872483670366, EEG2: 43.447444530575176


In [13]:
import pickle
import pandas as pd
import numpy as np
from neurodsp.spectral import compute_spectrum

def find_continuous_nrem_segments(sleep_stages):
    nrem_mask = sleep_stages['sleepStage'] == 2
    continuous_segments = []
    start_idx = None
    for i in range(len(nrem_mask)):
        if nrem_mask[i] and start_idx is None:
            start_idx = i
        elif not nrem_mask[i] and start_idx is not None:
            end_idx = i - 1
            continuous_segments.append((start_idx, end_idx))
            start_idx = None
    if start_idx is not None:
        continuous_segments.append((start_idx, len(nrem_mask) - 1))
    return continuous_segments

def compute_epoch_power(eeg_data, start_idx, end_idx, fs=512, freq_range=(0.5, 50)):
    eeg1_epoch = eeg_data['EEG1'][start_idx:end_idx+1]
    eeg2_epoch = eeg_data['EEG2'][start_idx:end_idx+1]
    
    f, Pxx_eeg1 = compute_spectrum(eeg1_epoch, fs, method='welch', nperseg=fs*2)
    f, Pxx_eeg2 = compute_spectrum(eeg2_epoch, fs, method='welch', nperseg=fs*2)
    
    freq_mask = (f >= freq_range[0]) & (f <= freq_range[1])
    
    power_eeg1 = np.mean(Pxx_eeg1[freq_mask])
    power_eeg2 = np.mean(Pxx_eeg2[freq_mask])
    
    return power_eeg1, power_eeg2

def compute_baseline_power(eeg_data, sleep_stages, fs=512, freq_range=(0.5, 50)):
    segments = find_continuous_nrem_segments(sleep_stages)
    power_eeg1_list = []
    power_eeg2_list = []
    
    for start_idx, end_idx in segments:
        power_eeg1, power_eeg2 = compute_epoch_power(eeg_data, start_idx, end_idx, fs, freq_range)
        power_eeg1_list.append(power_eeg1)
        power_eeg2_list.append(power_eeg2)
    
    baseline_power_eeg1 = np.mean(power_eeg1_list)
    baseline_power_eeg2 = np.mean(power_eeg2_list)
    
    
    return baseline_power_eeg1, baseline_power_eeg2


baseline_power_eeg1, baseline_power_eeg2 = compute_baseline_power(eeg_data, sleep_df)
print(f"Baseline NREM Power - EEG1: {baseline_power_eeg1}, EEG2: {baseline_power_eeg2}")

/Users/Volka/miniconda3/envs/fooof_venv/lib/python3.9/site-packages/neurodsp/spectral/power.py:181: UserWarning: nperseg = 1024 is greater than input length  = 512, using nperseg = 512
  freqs, _, spg = spectrogram(sig, fs, window, nperseg, noverlap)


Baseline NREM Power - EEG1: 62.08843150135539, EEG2: 43.52586578966183


In [1]:
import pickle
import pandas as pd
import numpy as np
from neurodsp.spectral import compute_spectrum

def load_pickle(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

def load_csv(file_path):
    return pd.read_csv(file_path)

def find_continuous_nrem_segments(sleep_stages):
    nrem_mask = sleep_stages['sleepStage'] == 2
    continuous_segments = []
    start_idx = None
    for i in range(len(nrem_mask)):
        if nrem_mask[i] and start_idx is None:
            start_idx = i
        elif not nrem_mask[i] and start_idx is not None:
            end_idx = i - 1
            continuous_segments.append((start_idx, end_idx))
            start_idx = None
    if start_idx is not None:
        continuous_segments.append((start_idx, len(nrem_mask) - 1))
    return continuous_segments

def compute_epoch_power(eeg_data, start_idx, end_idx, fs=512, freq_range=(0.5, 50)):
    eeg1_epoch = eeg_data['EEG1'][start_idx:end_idx+1]
    eeg2_epoch = eeg_data['EEG2'][start_idx:end_idx+1]
    
    f, Pxx_eeg1 = compute_spectrum(eeg1_epoch, fs, method='welch', nperseg=fs*2)
    f, Pxx_eeg2 = compute_spectrum(eeg2_epoch, fs, method='welch', nperseg=fs*2)
    
    freq_mask = (f >= freq_range[0]) & (f <= freq_range[1])
    
    power_eeg1 = np.mean(Pxx_eeg1[freq_mask])
    power_eeg2 = np.mean(Pxx_eeg2[freq_mask])
    
    return power_eeg1, power_eeg2

def compute_baseline_power(eeg_data, sleep_stages, fs=512, freq_range=(0.5, 50)):
    segments = find_continuous_nrem_segments(sleep_stages)
    power_eeg1_list = []
    power_eeg2_list = []
    
    for start_idx, end_idx in segments:
        power_eeg1, power_eeg2 = compute_epoch_power(eeg_data, start_idx, end_idx, fs, freq_range)
        power_eeg1_list.append(power_eeg1)
        power_eeg2_list.append(power_eeg2)
    
    baseline_power_eeg1 = np.mean(power_eeg1_list)
    baseline_power_eeg2 = np.mean(power_eeg2_list)

    baseline_power = (baseline_power_eeg1 + baseline_power_eeg2) / 2
    
    return baseline_power_eeg1, baseline_power_eeg2, baseline_power

def main(eeg_pickle_path, sleep_stage_csv_path):
    eeg_data = load_pickle(eeg_pickle_path)
    sleep_stages = load_csv(sleep_stage_csv_path)
    baseline_power_eeg1, baseline_power_eeg2 = compute_baseline_power(eeg_data, sleep_stages)
    average_baseline_power = np.mean([baseline_power_eeg1, baseline_power_eeg2])
    print(f"Baseline NREM Power - EEG1: {baseline_power_eeg1}, EEG2: {baseline_power_eeg2}")
    print(f"Average Baseline NREM Power: {average_baseline_power}")
    return average_baseline_power

# Example usage
average_baseline_power = main('/Volumes/harris/somnotate/to_score_set/pickle_eeg_signal/eeg_data_sub-010_ses-01_recording-01.pkl', 
                              '/Volumes/harris/somnotate/to_score_set/vis_back_to_csv/automated_state_annotationoutput_sub-010_ses-01_recording-01_time-0-69h_512Hz.csv')

/var/folders/sy/1pj77pp55jgbyxgsst2fw9rr0000gn/T/ipykernel_42075/11753367.py:8: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.load(f)
/Users/Volka/miniconda3/envs/fooof_venv/lib/python3.9/site-packages/neurodsp/spectral/power.py:181: UserWarning: nperseg = 1024 is greater than input length  = 512, using nperseg = 512
  freqs, _, spg = spectrogram(sig, fs, window, nperseg, noverlap)


ValueError: too many values to unpack (expected 2)

In [ ]:
import pickle
import pandas as pd
import numpy as np
from neurodsp.spectral import compute_spectrum
import matplotlib.pyplot as plt

def load_pickle(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

def load_csv(file_path):
    return pd.read_csv(file_path)

def find_continuous_nrem_segments(sleep_stages):
    nrem_mask = sleep_stages['sleepStage'] == 2
    continuous_segments = []
    start_idx = None
    for i in range(len(nrem_mask)):
        if nrem_mask[i] and start_idx is None:
            start_idx = i
        elif not nrem_mask[i] and start_idx is not None:
            end_idx = i - 1
            continuous_segments.append((start_idx, end_idx))
            start_idx = None
    if start_idx is not None:
        continuous_segments.append((start_idx, len(nrem_mask) - 1))
    return continuous_segments

def compute_epoch_power(eeg_data, start_idx, end_idx, fs=512, freq_range=(0.5, 4)):
    eeg1_epoch = eeg_data['EEG1'][start_idx:end_idx]
    eeg2_epoch = eeg_data['EEG2'][start_idx:end_idx]
    
    f, Pxx_eeg1 = compute_spectrum(eeg1_epoch, fs, method='welch', nperseg=fs*2)
    f, Pxx_eeg2 = compute_spectrum(eeg2_epoch, fs, method='welch', nperseg=fs*2)
    
    freq_mask = (f >= freq_range[0]) & (f <= freq_range[1])
    
    delta_power_eeg1 = np.mean(Pxx_eeg1[freq_mask])
    delta_power_eeg2 = np.mean(Pxx_eeg2[freq_mask])
    
    return delta_power_eeg1, delta_power_eeg2

def compute_baseline_power(eeg_data, sleep_stages, fs=512, freq_range=(0.5, 50)):
    segments = find_continuous_nrem_segments(sleep_stages)
    power_eeg1_list = []
    power_eeg2_list = []
    
    for start_idx, end_idx in segments:
        power_eeg1, power_eeg2 = compute_epoch_power(eeg_data, start_idx, end_idx, fs, freq_range)
        power_eeg1_list.append(power_eeg1)
        power_eeg2_list.append(power_eeg2)
    
    baseline_power_eeg1 = np.mean(power_eeg1_list)
    baseline_power_eeg2 = np.mean(power_eeg2_list)
    
    return baseline_power_eeg1, baseline_power_eeg2

def segment_epochs(sleep_stages, epoch_length=10, fs=512):
    epoch_samples = epoch_length * fs
    epochs = []
    for start in range(0, len(sleep_stages), epoch_samples):
        end = start + epoch_samples
        if end <= len(sleep_stages):
            epoch = sleep_stages[start:end]
            if all(epoch['sleepStage'] == 2):
                epochs.append((start, end))
    return epochs

def normalize_power(delta_power, baseline_power):
    return delta_power / baseline_power

def extract_epoch_exponents(fg_models, epochs):
    """Extract exponents for specific NREM epochs, 1 exponent per 10s."""
    exps_eeg1 = fg_models['EEG1'].get_params('aperiodic_params', 'exponent')
    exps_eeg2 = fg_models['EEG2'].get_params('aperiodic_params', 'exponent')
    
    epoch_exps = []
    samples_per_exp = 5120  # 10 seconds * 512 Hz
    
    for start_idx, end_idx in epochs:
        # Convert sample indices to exponent index (1 exp per 10s)
        exp_idx = start_idx // samples_per_exp
        
        # Get exponents for this epoch
        epoch_exp_eeg1 = exps_eeg1[exp_idx]
        epoch_exp_eeg2 = exps_eeg2[exp_idx]
        epoch_exps.append(np.mean([epoch_exp_eeg1, epoch_exp_eeg2]))
    
    return np.array(epoch_exps)

def main(eeg_pickle_path, sleep_stage_csv_path, fg_models_pickle_path):
    eeg_data = load_pickle(eeg_pickle_path)
    sleep_stages = load_csv(sleep_stage_csv_path)
    fg_models = load_pickle(fg_models_pickle_path)
    
    baseline_power_eeg1, baseline_power_eeg2 = compute_baseline_power(eeg_data, sleep_stages)
    average_baseline_power = np.mean([baseline_power_eeg1, baseline_power_eeg2])
    
    # Get valid NREM epochs
    epochs = segment_epochs(sleep_stages)
    
    # Calculate normalized delta powers for valid epochs
    normalized_delta_powers = []
    for start_idx, end_idx in epochs:
        delta_power_eeg1, delta_power_eeg2 = compute_epoch_power(eeg_data, start_idx, end_idx)
        avg_delta_power = np.mean([delta_power_eeg1, delta_power_eeg2])
        normalized_power = normalize_power(avg_delta_power, average_baseline_power)
        normalized_delta_powers.append(normalized_power)
    
    # Get exponents for the same epochs
    epoch_exponents = extract_epoch_exponents(fg_models, epochs)

    # Check lengths match
    if len(normalized_delta_powers) != len(epoch_exponents):
        print(f"Error: Length mismatch - Delta powers: {len(normalized_delta_powers)}, Exponents: {len(epoch_exponents)}")
        return
    
    plt.figure(figsize=(10, 6))
    plt.scatter(normalized_delta_powers, epoch_exponents, alpha=0.5)
    plt.xlabel('Normalized Delta Power')
    plt.ylabel('Average Exponent Value')
    plt.title('Normalized Delta Power vs Exponent Value (NREM Epochs)')
    plt.show()

# Example usage
# main('path/to/eeg_data.pkl', 'path/to/sleep_stages.csv', 'path/to/fg_models.pkl')

In [2]:
import pickle
import pandas as pd
import numpy as np
from neurodsp.spectral import compute_spectrum
import matplotlib.pyplot as plt

def find_continuous_nrem_segments(sleep_stages):
    nrem_mask = sleep_stages['sleepStage'] == 2
    continuous_segments = []
    start_idx = None
    for i in range(len(nrem_mask)):
        if nrem_mask[i] and start_idx is None:
            start_idx = i
        elif not nrem_mask[i] and start_idx is not None:
            end_idx = i - 1
            continuous_segments.append((start_idx, end_idx))
            start_idx = None
    if start_idx is not None:
        continuous_segments.append((start_idx, len(nrem_mask) - 1))
    return continuous_segments

def compute_epoch_power(eeg_data, start_idx, end_idx, fs=512, freq_range=(0.5, 4)):
    eeg1_epoch = eeg_data['EEG1'][start_idx:end_idx]
    eeg2_epoch = eeg_data['EEG2'][start_idx:end_idx]
    
    f, Pxx_eeg1 = compute_spectrum(eeg1_epoch, fs, method='welch', nperseg=fs*2)
    f, Pxx_eeg2 = compute_spectrum(eeg2_epoch, fs, method='welch', nperseg=fs*2)
    
    freq_mask = (f >= freq_range[0]) & (f <= freq_range[1])
    
    delta_power_eeg1 = np.mean(Pxx_eeg1[freq_mask])
    delta_power_eeg2 = np.mean(Pxx_eeg2[freq_mask])
    
    return delta_power_eeg1, delta_power_eeg2

def compute_baseline_power(eeg_data, sleep_stages, fs=512, freq_range=(0.5, 50)):
    segments = find_continuous_nrem_segments(sleep_stages)
    power_eeg1_list = []
    power_eeg2_list = []
    
    for start_idx, end_idx in segments:
        power_eeg1, power_eeg2 = compute_epoch_power(eeg_data, start_idx, end_idx, fs, freq_range)
        power_eeg1_list.append(power_eeg1)
        power_eeg2_list.append(power_eeg2)
    
    baseline_power_eeg1 = np.mean(power_eeg1_list)
    baseline_power_eeg2 = np.mean(power_eeg2_list)
    
    return baseline_power_eeg1, baseline_power_eeg2

def segment_epochs(sleep_stages, epoch_length=10, fs=512):
    epoch_samples = epoch_length * fs
    epochs = []
    for start in range(0, len(sleep_stages), epoch_samples):
        end = start + epoch_samples
        if end <= len(sleep_stages):
            epoch = sleep_stages[start:end]
            if all(epoch['sleepStage'] == 2):
                epochs.append((start, end))
    return epochs

def normalize_power(delta_power, baseline_power):
    return delta_power / baseline_power

def extract_epoch_exponents(fg_models, epochs):
    """Extract exponents for specific NREM epochs, 1 exponent per 10s."""
    exps_eeg1 = fg_models['EEG1'].get_params('aperiodic_params', 'exponent')
    exps_eeg2 = fg_models['EEG2'].get_params('aperiodic_params', 'exponent')
    
    epoch_exps = []
    samples_per_exp = 5120  # 10 seconds * 512 Hz
    
    for start_idx, end_idx in epochs:
        # Convert sample indices to exponent index (1 exp per 10s)
        exp_idx = start_idx // samples_per_exp
        
        # Get exponents for this epoch
        epoch_exp_eeg1 = exps_eeg1[exp_idx]
        epoch_exp_eeg2 = exps_eeg2[exp_idx]
        epoch_exps.append(np.mean([epoch_exp_eeg1, epoch_exp_eeg2]))
    
    return np.array(epoch_exps)

def main(eeg_pickle_path, sleep_stage_csv_path, fg_models_pickle_path):
    fg_models = load_pickle(fg_models_pickle_path)
    
    baseline_power_eeg1, baseline_power_eeg2 = compute_baseline_power(eeg_data, sleep_stages)
    average_baseline_power = np.mean([baseline_power_eeg1, baseline_power_eeg2])
    
    # Get valid NREM epochs
    epochs = segment_epochs(sleep_stages)
    
    # Calculate normalized delta powers for valid epochs
    normalized_delta_powers = []
    for start_idx, end_idx in epochs:
        delta_power_eeg1, delta_power_eeg2 = compute_epoch_power(eeg_data, start_idx, end_idx)
        avg_delta_power = np.mean([delta_power_eeg1, delta_power_eeg2])
        normalized_power = normalize_power(avg_delta_power, average_baseline_power)
        normalized_delta_powers.append(normalized_power)
    
    # Get exponents for the same epochs
    epoch_exponents = extract_epoch_exponents(fg_models, epochs)

    # Check lengths match
    if len(normalized_delta_powers) != len(epoch_exponents):
        print(f"Error: Length mismatch - Delta powers: {len(normalized_delta_powers)}, Exponents: {len(epoch_exponents)}")
        return
    
    plt.figure(figsize=(10, 6))
    plt.scatter(normalized_delta_powers, epoch_exponents, alpha=0.5)
    plt.xlabel('Normalized Delta Power')
    plt.ylabel('Average Exponent Value')
    plt.title('Normalized Delta Power vs Exponent Value (NREM Epochs)')
    plt.show()

# Example usage
main(eeg_data, sleep_df, '/Volumes/harris/volkan/fooof/fooof_results/sub-010_ses-01_recording-01_fooof.pkl')

NameError: name 'eeg_data' is not defined